In [15]:
#import all necessary libraries

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [40]:
"""
pytorch implementation of Net Class, extends the nn.Module 
which is the building block for all pytorch models
"""

class Net(nn.Module):
    def __init__(self):                                #        Channel dim        |       Receptive Field  
        super(Net, self).__init__()                    #    input       output     |     input       output
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)    #     28           28       |       1           3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)   #     28           28       |       3           5
        self.pool1 = nn.MaxPool2d(2, 2)                #     28           14       |       5          10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)  #     14           14       |      10          12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #     14           14       |      12          14
        self.pool2 = nn.MaxPool2d(2, 2)                #     14            7       |      14          28
        self.conv5 = nn.Conv2d(256, 512, 3)            #      7            5       |      28          30
        self.conv6 = nn.Conv2d(512, 1024, 3)           #      5            3       |      30          32
        self.conv7 = nn.Conv2d(1024, 10, 3)            #      3            1       |      32          34

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))  # First Block edges and gradients
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))  # Second Block Textures and Patterns
        x = F.relu(self.conv6(F.relu(self.conv5(x))))              # Third Block Parts of Objects
        x = self.conv7(x) #F.relu(self.conv7(x))
        x = x.view(-1, 10)  
        return F.log_softmax(x)   # Softmax function which forces the network to predict a class

In [41]:
from torchsummary import summary  # gives nice visualization for the network
use_cuda = torch.cuda.is_available() # check if GPU is available
device = torch.device("cuda" if use_cuda else "cpu") # initialize cuda device
model = Net().to(device) # load the model onto GPU
summary(model, input_size=(1, 28, 28)) # Visualize the network

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [42]:
torch.manual_seed(1) # set random seed to get reproducable results
batch_size = 128

# pin memory helps to lod data onto GPU qickly while training
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# define train dataloader with all parameters
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)) # do image normalization with mean and std of entire dataset
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# define test dataloader with all parameters
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [43]:
from tqdm import tqdm  # library to show progress bar

# train function
def train(model, device, train_loader, optimizer, epoch):
    model.train() # sets the model on training mode
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) # load the batch onto device(GPU)
        optimizer.zero_grad() # set the gradients manually to zero, since gradients are cumulative we need to set it to zero before avery batch
        output = model(data) # train 1 batch
        loss = F.nll_loss(output, target) # compute negative log likelihood loss
        loss.backward() # compute gradients
        optimizer.step() # back propagate
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

# test function
def test(model, device, test_loader):
    model.eval() # sets the model on prediction mode
    test_loss = 0
    correct = 0
    with torch.no_grad(): # disables back propagation
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data) # predict
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [44]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # optimizer Stochastic Gradient Descend

for epoch in range(1, 2): # epochs
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.033012863248586655 batch_id=468: 100%|██████████| 469/469 [00:09<00:00, 48.93it/s]



Test set: Average loss: 0.0672, Accuracy: 9789/10000 (98%)

